In [3]:
"""
this script is designed to processing pdf data for embedding
"""

import PyPDF2
import textwrap
import tiktoken
import nltk


In [ ]:
# This part of the spell disables SSL certificate verification.
ssl._create_default_https_context = ssl._create_unverified_context

# Download the NLTK data required for sentence tokenization
nltk.download('punkt')

In [6]:
# specify the local path to the pdf files
f23_pdf_path = '/Users/sining/Library/CloudStorage/GoogleDrive-sxw924@case.edu/My Drive/Research/pedagogical research/GenerativeAI for teaching and learning/GenAI in Essay Writing/NudgeEssayResearch/clean_file/F23'

In [8]:

# open the pdf file
pdf_file = open(f23_pdf_path + '/essay1.pdf', 'rb')

# Create a PDF reader object
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Get the number of pages
num_pages = len(pdf_reader.pages)

# Extract text from each page
text = ''
for page_num in range(num_pages):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

# Close the PDF file
pdf_file.close()

In [9]:
# Load the encoding for the language model
encoding = tiktoken.get_encoding('gpt2')

def split_text_into_batches(text, max_tokens=50):
    """
    Split a long text into batches based on the number of tokens,
    ensuring each batch contains complete sentences.
    """
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)
    
    # Initialize an empty list to store the batches
    batches = []
    
    # Initialize the current batch with an empty list
    current_batch = []
    current_batch_tokens = []
    
    for sentence in sentences:
        # Encode the sentence into tokens
        sentence_tokens = encoding.encode(sentence)
        
        # If adding the sentence to the current batch would make it too long,
        # add the current batch to the list of batches and start a new batch
        if len(current_batch_tokens) + len(sentence_tokens) > max_tokens:
            batches.append(encoding.decode(current_batch_tokens))
            current_batch = [sentence]
            current_batch_tokens = sentence_tokens
        else:
            # Otherwise, add the sentence to the current batch
            current_batch.append(sentence)
            current_batch_tokens.extend(sentence_tokens)
    
    # Add the last batch to the list of batches
    if current_batch_tokens:
        batches.append(encoding.decode(current_batch_tokens))
    
    return batches

In [10]:
batches = split_text_into_batches(text)

for batch in batches:
    print(f"Batch length (tokens): {len(encoding.encode(batch))}")
    print(batch)
    print("-" * 50)

Batch length (tokens): 42
Ethical Implication of Nudges  
Introduction 
Nudging as a term becam e popular becaus e of Richard Thaler’s book, “ Nudge ”.
--------------------------------------------------
Batch length (tokens): 36
A nudg e is any 
deliberat e structuring of choices that can predict behavior.The nudg e makes decisions easier without 
removing any options.
--------------------------------------------------
Batch length (tokens): 34
Choic e architectur e changes how a decision maker perceives options.For 
example, changing a tipping system to default the consumer paying 20% gratuity.
--------------------------------------------------
Batch length (tokens): 26
This creates mor e tips 
than the old system where default is tipping 0% (Zarrabian, 2019) .
--------------------------------------------------
Batch length (tokens): 25
The new choic e makes a person feel 
that they take something away from the server instead of giving something extra.
--------------------------------

In [12]:
# export the batch to a txt file, save in the folder rawtext. name as the original file nam
with open('rawtext/essay1.txt', 'w') as file:
    for batch in batches:
        file.write(batch)
        file.write('\n')
        file.write("-" * 50)
        file.write('\n')
        file.write('\n')
    file.close()